In [1]:
import pandas as pd
import pydeck as pdk

In [2]:
df = pd.read_csv('/Users/carboni/Downloads/Data Engineering Crist.csv')

In [3]:
df.head()

,ID Artiste,ouvre,Nom,Prénom,full_name,Nom collectivité,Tous les noms,Fonction,Type,Biographie,...,Naissance_Century,Nationalité,continent,Sexe,Statut notice,ID Sikart,ID ULAN,ID GND,URL,Comments
0,37613.0,NaN,Pannier,Jacques Étienne,Jacques Etienne Pannier,NaN,Jacques Étienne Pannier,"graveur, peintre",Individu,"— Date de naissance, 17.11.1802, France, Paris...",...,19.0,France,Europe,M,NaN,NaN,NaN,NaN,NaN,NaN
1,129.0,NaN,Paolini,Giulio,Giulio Paolini,NaN,Giulio Paolini,NaN,Individu,"— Date de naissance, 1940, date imprécise : 19...",...,20.0,Italy,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60015.0,NaN,Paolozzi,Eduardo,Eduardo Paolozzi,NaN,Eduardo Paolozzi,NaN,Individu,"— Date de naissance, 7.3.1924, date imprécise ...",...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64733.0,NaN,Papageorgiu,Dimitri,Dimitri Papageorgiu,NaN,Dimitri Papageorgiu; [Papagueorguiu ],NaN,Individu,"— Date de naissance, 08.08.1928, Grèce, Staghi...",...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,419290.0,NaN,Papavassiliou,Petros,Petros Papavassiliou,NaN,Petros Papavassiliou,NaN,Individu,"— Date de naissance, 06.12.1928, date imprécis...",...,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns.unique()

Index(['ID Artiste', 'ouvre', 'Nom', 'Prénom', 'full_name', 'Nom collectivité',
       'Tous les noms', 'Fonction', 'Type', 'Biographie', 'Date de début',
       'Période d'activ./prod', 'Naissance', 'Naissance_Location2',
       'Naissance_Location2FR', 'Match between naissance and nationalitzy',
       'Naissance_Location1', 'Naissance_Location1_count', 'Lat_location_corr',
       'Long_location_corr', 'museum_lat', 'museum_long',
       'Naissance_Location_Alternative', 'Mariage', 'Décès',
       'Lat_location_death', 'Long_location_death', 'Décès_Day',
       'deces_datetime', 'Décès_Year', 'Naissance_Day', 'naissance_datetime',
       'Naissance_Year', 'Naissance_Century', 'Nationalité', 'continent',
       'Sexe', 'Statut notice', 'ID Sikart', 'ID ULAN', 'ID GND', 'URL',
       'Comments'],
      dtype='object')

In [5]:
#change the name of the column!
df['source'] = df.apply(lambda row: [row['Long_location_corr'], row['Lat_location_corr']], axis=1)
df['target'] = df.apply(lambda row: [row['museum_long'], row['museum_lat']], axis=1)

In [6]:
df['arc_id'] = df.apply(lambda row: f"{row['source']}-{row['target']}", axis=1)

In [7]:
arc_counts = df['arc_id'].value_counts().reset_index()
arc_counts.columns = ['arc_id', 'arc_count']

In [8]:
df = df.merge(arc_counts, on='arc_id')

In [9]:
GREEN_RGB = [200, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]

In [25]:
arc_layer = pdk.Layer(
    "ArcLayer",
    df,
    #get_width="arc_count * 2", 
    get_source_position="source",
    get_target_position="target",
    get_tilt=10,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    get_height=2,  
    pickable=True,
    auto_highlight=True
)

In [21]:
view_state = pdk.ViewState(latitude=df['Lat_location_corr'].mean(), longitude=df['Long_location_corr'].mean(), zoom=1, bearing=15, pitch=45)

In [22]:
tooltips = {
    "html": "<b>Origin:</b> {Naissance_Location1}<br/><b>Destination:</b> {target}<br/><b>Count:</b> {arc_count}",
    "style": {"backgroundColor": "steelblue", "color": "white"}
}

In [26]:
r = pdk.Deck(layers=[arc_layer], initial_view_state=view_state, tooltip=tooltips, map_provider="carto", map_style="dark")

In [27]:
r.to_html('arcs_between_locations.html')

In [34]:
df["source_palladio"] = df["source"].apply(lambda x: ', '.join(map(str, x)))

In [35]:
df["destination_palladio"] = df["source"].apply(lambda x: ', '.join(map(str, x)))

In [36]:
df.to_csv('arc_kepler.csv', index=False)